In [2]:
import gensim

In [3]:
class targetCorpus:
    def __iter__(self):
        with open("hits.dat") as hits:
            for line in hits:
                yield line.split()
def takelast(x):
    return x[-1]


In [4]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models import EnsembleLda

In [5]:
import gensim.downloader as api
from gensim.corpora import Dictionary
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = api.load('text8')

In [6]:
dictionary = Dictionary()
dictionary.add_documents(targetCorpus())
for doc in docs:
    dictionary.add_documents([[lemmatizer.lemmatize(token) for token in doc]])
dictionary.filter_extremes(no_below=20, no_above=0.5)

corpus = [dictionary.doc2bow(doc) for doc in docs]

In [7]:
from gensim.models import LdaModel
topic_model_class = LdaModel
ensemble_workers = 4
num_models = 8
distance_workers = 4
num_topics = 200
passes = 2

In [8]:
from gensim.models import EnsembleLda
ensemble = EnsembleLda(
    corpus=corpus,
    id2word=dictionary,
    num_topics=num_topics,
    passes=passes,
    num_models=num_models,
    topic_model_class=LdaModel,
    ensemble_workers=ensemble_workers,
    distance_workers=distance_workers
)

print(len(ensemble.ttda))
print(len(ensemble.get_topics()))

1600
5


In [12]:
ensemble.recluster(eps=0.1)

print(len(ensemble.get_topics()))

5


In [14]:
elda = ensemble
with open("hits.dat") as hits:
    for line in hits:
        c = [dictionary.doc2bow(line.split())]
        elda.add_model(LdaModel(c, id2word=dictionary, num_topics=3))
elda.recluster()

In [77]:
i = 22
i-=1
model = elda
c = [dictionary.doc2bow(doc) for doc in targetCorpus()]
top_topics = elda[c[i]] # [(topic_id, prob)]
print(len(ensemble.ttda))
print(len(ensemble.get_topics()))
#print(phrased_doc[i])
####
top_topics.sort(key=takelast, reverse=True)
print(top_topics)
for topic in top_topics:
    print('topic {}'.format(topic))
    print(model.print_topics(topic[0]))
    i+=1
#     topic_term_distribution = model.get_topic_terms(topic[0])
#     for term in topic_term_distribution:
#         print('{}, {}'.format(dictionary[term[0]], term[1]))
#     print('prob: {}'.format(topic[-1]))

448
2
[(1, 0.7519974), (0, 0.24800262)]
topic (1, 0.7519974)
[(1, '0.029*"as" + 0.001*"times" + 0.001*"player" + 0.001*"km" + 0.001*"team" + 0.001*"college" + 0.001*"means" + 0.001*"soviet" + 0.001*"al" + 0.001*"league"')]
topic (0, 0.24800262)
[(1, '0.029*"as" + 0.001*"times" + 0.001*"player" + 0.001*"km" + 0.001*"team" + 0.001*"college" + 0.001*"means" + 0.001*"soviet" + 0.001*"al" + 0.001*"league"'), (0, '0.000*"indicate" + 0.000*"responding" + 0.000*"precisely" + 0.000*"respond" + 0.000*"interpreted" + 0.000*"incapable" + 0.000*"keyword" + 0.000*"output" + 0.000*"grained" + 0.000*"describe"')]
